In [1]:
# Import all the necessary modules/packages
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, explode, to_timestamp, expr, lit 
import boto3
import os
import glob
import shutil

In [2]:
# Start pySpark session
spark = SparkSession.builder \
                    .appName("twitch_emotes_analytics_project") \
                    .master("local[*]") \
                    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/07/01 12:56:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/07/01 12:56:18 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
25/07/01 12:56:18 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
25/07/01 12:56:18 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
25/07/01 12:56:18 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check whether configuring an appropriate binding address.
25/07/01 12:56:18 WARN Utils: Service 'sparkDriver' could not bind on a random free port. You may check

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.net.BindException: Can't assign requested address: Service 'sparkDriver' failed after 16 retries (on a random free port)! Consider explicitly setting the appropriate binding address for the service 'sparkDriver' (for example spark.driver.bindAddress for SparkDriver) to the correct binding address.
	at java.base/sun.nio.ch.Net.bind0(Native Method)
	at java.base/sun.nio.ch.Net.bind(Net.java:459)
	at java.base/sun.nio.ch.Net.bind(Net.java:448)
	at java.base/sun.nio.ch.ServerSocketChannelImpl.bind(ServerSocketChannelImpl.java:227)
	at io.netty.channel.socket.nio.NioServerSocketChannel.doBind(NioServerSocketChannel.java:141)
	at io.netty.channel.AbstractChannel$AbstractUnsafe.bind(AbstractChannel.java:562)
	at io.netty.channel.DefaultChannelPipeline$HeadContext.bind(DefaultChannelPipeline.java:1334)
	at io.netty.channel.AbstractChannelHandlerContext.invokeBind(AbstractChannelHandlerContext

In [3]:
spark

NameError: name 'spark' is not defined

In [ ]:
# Input from user
channel_name = input("Please Enter the Channel Name: ")
chat_years_input = input("Please Enter the Chat Years (comma-separated): ")
# Convert input to list of years
chat_years = [year.strip() for year in chat_years_input.split(",") if year.strip().isdigit() and len(year.strip()) == 4]
print(chat_years)

In [ ]:
# S3 setup
bucket = "twitch-emotes-analytics-project"
s3 = boto3.client("s3")

In [ ]:
# Local paths
local_path = "/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/processed_silver/"
output_local_path = "/Users/hari14/Desktop/PHD/twitch_emotes_analytics_project/data/processed_silver/"

In [ ]:
# order raw s3 files
def order_files(json_files):
    try:
        return sorted(
            json_files,
            key=lambda x: int(os.path.basename(x).replace(".json", "").split("_")[-2])
        )
    except (IndexError, ValueError) as e:
        raise ValueError(f"Filename parsing failed: {e}")

In [ ]:
# processing loop
for chat_year in chat_years:
    s3_key_prefix = f"data/raw_bronze/{channel_name}/{chat_year}/"
    response = s3.list_objects_v2(Bucket=bucket, Prefix=s3_key_prefix)
    contents = response.get("Contents", [])
    
    if not contents:
        print(f"🚫 No files found at S3 path: {s3_key_prefix}")
        continue

    json_files = [i["Key"] for i in contents]
    sorted_json_files = order_files(json_files)

    for i, key in enumerate(sorted_json_files):
        file_name = os.path.basename(key)
        local_file_path = os.path.join(local_path, file_name)
        temp_processed_path = os.path.join(output_local_path, "temp_processed")

        # Extract VOD ID from filename
        file_vod_id = file_name.replace(".json", "").split("_")[-1]

        try:
            # Download
            s3.download_file(Bucket=bucket, Key=key, Filename=local_file_path)
            print(f"📥 file {i+1}: {file_name} downloaded and being processed")

            # Read JSON
            df = spark.read.format("json").option("multiLine", True).load(local_file_path)
            df_filtered = df.filter(col("emotes").isNotNull())
            df_exploded = df_filtered.withColumn("emote", explode("emotes"))

            # Add columns
            df_badges = df_exploded.withColumn("i_badge_names", expr("transform(author.badges, x -> x.name)")) \
                                   .withColumn("i_badge_titles", expr("transform(author.badges, x -> x.title)")) \
                                   .withColumn("i_badge_versions", expr("transform(author.badges, x -> x.version)"))

            # Final select with VOD ID added
            df_panel = df_badges.select(
                col("author.id").cast("long").alias("i_user_id"),
                col("author.name").alias("i_username"),
                col("author.colour").alias("i_display_color"),
                col("i_badge_names"),
                col("i_badge_titles"),
                col("i_badge_versions").cast("array<string>"),
                lit(None).cast("string").alias("i_user_status"),
                lit(channel_name).alias("j_streamer"),
                col("emote.name").alias("k_emote_name"),
                to_timestamp((col("timestamp") / 1_000_000).cast("timestamp")).alias("t_timestamp"),
                col("time_text").alias("t_time_text"),
                col("time_in_seconds").cast("long").alias("t_seconds"),
                lit(file_vod_id).alias("vod_id"),
                col("message").alias("chat_message")
            )

            # Write parquet
            df_panel.coalesce(1).write.mode("overwrite").parquet(temp_processed_path)
            parquet_files = glob.glob(os.path.join(temp_processed_path, "part-*.parquet"))
            if not parquet_files:
                raise FileNotFoundError("No parquet file found in temp_processed folder")

            # Upload to S3
            parquet_file_path = parquet_files[0]
            s3_output_key = f"data/processed_silver/{channel_name}/{chat_year}/{file_name.replace('.json', '.parquet')}"
            s3.upload_file(parquet_file_path, bucket, s3_output_key)
            print(f"✅ Uploaded: s3://{bucket}/{s3_output_key}")

        except Exception as e:
            print(f"❌ Error processing file {file_name}: {e}")

        finally:
            # Cleanup
            if os.path.exists(temp_processed_path):
                shutil.rmtree(temp_processed_path)
            if os.path.exists(local_file_path):
                os.remove(local_file_path)
            print(f"🧹 Cleaned up local files for {file_name} (file {i+1}/{len(sorted_json_files)})")